In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import os
import re
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, LineTokenizer, RegexpTokenizer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()
import string
from nltk.stem import PorterStemmer, WordNetLemmatizer
stopwords_indonesia = stopwords.words('indonesian')
stop_words = set(stopwords.words('indonesian'))
import dataset

In [7]:
from nltk.tokenize import word_tokenize
result_politik = []

# loop untuk setiap sheet pada dataframe
for df in dataset.Politik_all_sheets:
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # periksa maksimum, minimum, dan rata-rata panjang jawaban dan panjang soal pada setiap kategori
    for column in ['Jawaban']:
        # hitung panjang jawaban/soal dalam jumlah kata
        lengths = df[column].apply(lambda x: len(word_tokenize(x)))
        max_len = lengths.max()
        min_len = lengths.min()
        avg_len = lengths.mean()
        result_politik.append({
            'Max Length': max_len,
            'Min Length': min_len,
            'Average Length': avg_len
        })

result_politik = pd.DataFrame(result_politik)

In [8]:
result_politik

,Max Length,Min Length,Average Length
0,127,2,28.703704
1,50,2,28.425926
2,679,3,48.425926
3,94,4,29.388889
4,444,2,38.388889
5,314,2,30.796296
6,282,1,46.018519
7,84,2,29.641509
8,154,2,32.549020
9,105,3,21.792453


In [9]:
max_length_answer_politik = result_politik['Max Length'].max()
min_length_answer_politik = result_politik['Min Length'].min()
average_length_answer_politik = result_politik['Average Length'].mean()

In [13]:
result_soal_politik = []

lengths = dataset.Politik_Soal['Soal '].apply(lambda x: len(word_tokenize(x)))
max_len = lengths.max()
min_len = lengths.min()
avg_len = lengths.mean()
result_soal_politik.append({
    'Max Length': max_len,
    'Min Length': min_len,
    'Average Length': avg_len
})
        
result_soal_politik = pd.DataFrame(result_soal_politik)

In [14]:
result_soal_politik

,Max Length,Min Length,Average Length
0,26,5,14.1


In [15]:
result_politik_case = []

k = 0
# loop untuk setiap sheet pada dataframe
for df in dataset.Politik_all_sheets:
    k = k + 1
    capital_words = {}
    lower_words = {}
    other_words = {}
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # periksa maksimum, minimum, dan rata-rata panjang jawaban dan panjang soal pada setiap kategori
    for token in df["Jawaban"]:
        if isinstance(token, str):
            if token.isupper():
                capital_words[token] = capital_words.get(token, 0) + 1
            elif token.islower():
                lower_words[token] = lower_words.get(token, 0) + 1
            else:
                other_words[token] = other_words.get(token, 0) + 1

    # Menampilkan hasil
    print(f'Soal {k}')
    print("Jumlah kata berhuruf kapital: ", len(capital_words))
    print("Jumlah kata berhuruf kecil: ", len(lower_words))
    print("Jumlah kata lainnya: ", len(other_words))
    print('\n')

Soal 1
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  25
Jumlah kata lainnya:  29


Soal 2
Jumlah kata berhuruf kapital:  4
Jumlah kata berhuruf kecil:  12
Jumlah kata lainnya:  38


Soal 3
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  14
Jumlah kata lainnya:  40


Soal 4
Jumlah kata berhuruf kapital:  1
Jumlah kata berhuruf kecil:  15
Jumlah kata lainnya:  38


Soal 5
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  23
Jumlah kata lainnya:  30


Soal 6
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  25
Jumlah kata lainnya:  27


Soal 7
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  24
Jumlah kata lainnya:  25


Soal 8
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  5
Jumlah kata lainnya:  46


Soal 9
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  18
Jumlah kata lainnya:  31


Soal 10
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  14
Jumlah kata lainnya:  39




In [16]:
result_politik_unrater = []

soal_politik = 0
# loop untuk setiap sheet pada dataframe
for df in dataset.Politik_all_sheets:
    soal_politik = soal_politik + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    unrated_answers = df[df[['Manual 1', 'Manual 2', 'Manual 3']].isnull().all(axis=1)]
    result_politik_unrater.append({
        'Soal': soal_politik,
        'Column': 'Jawaban',
        'Unrated Answers': len(unrated_answers)
    })

# tampilkan hasil analisis
for r in result_politik_unrater:
    print(f"Soal {r['Soal']} - Unrated Answers: {r['Unrated Answers']}")

Soal 1 - Unrated Answers: 0
Soal 2 - Unrated Answers: 0
Soal 3 - Unrated Answers: 0
Soal 4 - Unrated Answers: 0
Soal 5 - Unrated Answers: 0
Soal 6 - Unrated Answers: 0
Soal 7 - Unrated Answers: 0
Soal 8 - Unrated Answers: 0
Soal 9 - Unrated Answers: 0
Soal 10 - Unrated Answers: 0


In [17]:
j = 0
for df in dataset.Politik_all_sheets:
    j = j + 1
    df.rename(columns= {'Siswa ': 'Siswa', 'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Siswa', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # Mengecek duplikasi pada kolom jawaban
        
    duplicate_jawaban_rate_politik = df[df.duplicated(['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual'], keep=False)]
    if not duplicate_jawaban_rate_politik.empty:
        print(f"Duplikasi pada kolom soal {j}:")
        print(duplicate_jawaban_rate_politik[['Siswa', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']])
        print('\n')

Duplikasi pada kolom soal 6:
         Siswa           Jawaban  Manual 1  Manual 2  Manual 3  Rata Manual
No                                                                         
5     siswa_6   masyarakat kuno        5.0       0.0      10.0          5.0
20   siswa_26   masyarakat kuno        5.0       0.0      10.0          5.0


Duplikasi pada kolom soal 7:
         Siswa                                            Jawaban  Manual 1  \
No                                                                            
8     siswa_9   Hukum perdata adalah sekumpulan aturan hukum y...      80.0   
9    siswa_10   Hukum perdata ialah aturan-aturan hukum yang m...      70.0   
10   siswa_11   Hukum perdata ialah aturan-aturan hukum yang m...      70.0   
21   siswa_27   Hukum perdata adalah sekumpulan aturan hukum y...      80.0   

     Manual 2  Manual 3  Rata Manual  
No                                    
8        80.0      75.0    78.333333  
9        75.0      75.0    73.333333  
10   

In [18]:
total_punctuation = 0
total_stopword = 0
total_berimbuhan = 0
total_non_lemma = 0
soal_politik = 0

for df in dataset.Politik_all_sheets:
    soal_politik = soal_politik + 1
    freq_punctuation = 0
    freq_stopword = 0
    freq_berimbuhan = 0
    freq_non_lemma = 0
    
    # proses setiap jawaban siswa pada sheet
    for jawaban in df['Jawaban']:
        # hitung frekuensi punctuation
        freq_punctuation += sum([1 for char in jawaban if char in string.punctuation])
        
        # hitung frekuensi stopword
        tokenized = word_tokenize(jawaban)
        freq_stopword += sum([1 for word in tokenized if word in stop_words])
        
        # hitung frekuensi kata berimbuhan
        freq_berimbuhan += sum([1 for word in tokenized if '-' in word or 'nya' in word or 'ku' in word or 'mu' in word or 'lah' in word or 'kah' in word])
        
        # hitung frekuensi kata non-lemma
        stemmed = [stemmer.stem(word) for word in tokenized]
        freq_non_lemma += sum([1 for i in range(len(tokenized)) if tokenized[i] != stemmed[i]])
    
    # tambahkan frekuensi pada variabel total
    total_punctuation += freq_punctuation
    total_stopword += freq_stopword
    total_berimbuhan += freq_berimbuhan
    total_non_lemma += freq_non_lemma
    
    # tampilkan frekuensi untuk sheet saat ini
    print(f"Soal: {soal_politik}:")
    print(f"Frekuensi Punctuation: {freq_punctuation}")
    print(f"Frekuensi Stopword: {freq_stopword}")
    print(f"Frekuensi Kata Berimbuhan: {freq_berimbuhan}")
    print(f"Frekuensi Kata Non-Lemma: {freq_non_lemma}")
    print()
    
# tampilkan total frekuensi untuk seluruh file excel
print("Total:")
print(f"Frekuensi Punctuation: {total_punctuation}")
print(f"Frekuensi Stopword: {total_stopword}")
print(f"Frekuensi Kata Berimbuhan: {total_berimbuhan}")
print(f"Frekuensi Kata Non-Lemma: {total_non_lemma}")

Soal: 1:
Frekuensi Punctuation: 166
Frekuensi Stopword: 554
Frekuensi Kata Berimbuhan: 128
Frekuensi Kata Non-Lemma: 649

Soal: 2:
Frekuensi Punctuation: 250
Frekuensi Stopword: 533
Frekuensi Kata Berimbuhan: 219
Frekuensi Kata Non-Lemma: 745

Soal: 3:
Frekuensi Punctuation: 323
Frekuensi Stopword: 852
Frekuensi Kata Berimbuhan: 248
Frekuensi Kata Non-Lemma: 1425

Soal: 4:
Frekuensi Punctuation: 287
Frekuensi Stopword: 349
Frekuensi Kata Berimbuhan: 97
Frekuensi Kata Non-Lemma: 847

Soal: 5:
Frekuensi Punctuation: 287
Frekuensi Stopword: 716
Frekuensi Kata Berimbuhan: 206
Frekuensi Kata Non-Lemma: 852

Soal: 6:
Frekuensi Punctuation: 239
Frekuensi Stopword: 497
Frekuensi Kata Berimbuhan: 143
Frekuensi Kata Non-Lemma: 755

Soal: 7:
Frekuensi Punctuation: 278
Frekuensi Stopword: 943
Frekuensi Kata Berimbuhan: 523
Frekuensi Kata Non-Lemma: 1032

Soal: 8:
Frekuensi Punctuation: 188
Frekuensi Stopword: 468
Frekuensi Kata Berimbuhan: 146
Frekuensi Kata Non-Lemma: 805

Soal: 9:
Frekuensi Punc